In [1]:
import os
print(os.getcwd())

d:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction


## Alternative method using donwsampled PNG by PNG for feature extraction

### Feature extraction function

In [ ]:
import torch
import timm
from PIL import Image
from torchvision import transforms
from huggingface_hub import login

class HOptimusExtractorFiveCrop:
    def __init__(self, device='cuda'):
        # Login to Hugging Face (if needed)
        login(token="")

        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')

        self.model = timm.create_model(
            "hf-hub:bioptimus/H-optimus-1",
            pretrained=True,
            init_values=1e-5,
            dynamic_img_size=False
        ).to(self.device)
        self.model.eval()


        self.eval_transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize each crop to 224x224
            transforms.ToTensor(),
            transforms.Normalize(
                mean=(0.707223, 0.578729, 0.703617),
                std=(0.211883, 0.230117, 0.177517)
            )
        ])

        self.fivecrop_transform = transforms.Compose([
            transforms.FiveCrop(256),
            transforms.Lambda(lambda crops: torch.stack([self.eval_transform(c) for c in crops]))
        ])

    def extract_fivecrop_features(self, image: Image.Image | str) -> torch.Tensor:
        """
        Extract features from 5 crops of an image.
        Returns a tensor of shape (5, 1536)
        """
        if isinstance(image, str):
            image = Image.open(image).convert("RGB")
        else:
            image = image.convert("RGB")

        crops_tensor = self.fivecrop_transform(image).to(self.device)  # (5, 3, 224, 224)

        with torch.autocast(device_type=self.device.type, dtype=torch.float16):
            with torch.inference_mode():
                features = self.model(crops_tensor)  # (5, 1536)

        return features.cpu()

## Testing

In [2]:
# FiveCrop feature extraction function
import glob
import os
from tqdm import tqdm
import torch

# Set paths
patch_root = r"D:\Aamir Gulzar\KSA_project2\dataset\patch_data"
save_root = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\H_Optimus_1_fivecrop_Updated"

# Initialize extractor
extractor = HOptimusExtractorFiveCrop()

# Loop through all PNG patches
all_patch_paths = glob.glob(os.path.join(patch_root, "*", "*.png"))

for patch_path in tqdm(all_patch_paths):
    try:
        features = extractor.extract_fivecrop_features(patch_path)  # shape: (5, D)

        # Extract subfolder and filename
        relative_path = os.path.relpath(patch_path, patch_root)
        slide_folder, patch_filename = os.path.split(relative_path)
        patch_base = os.path.splitext(patch_filename)[0]  # remove .png

        # Create save path
        save_dir = os.path.join(save_root, slide_folder)
        os.makedirs(save_dir, exist_ok=True)

        save_path = os.path.join(save_dir, patch_base + ".pt")
        torch.save(features, save_path)
    
    except Exception as e:
        print(f"Error processing {patch_path}: {e}")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\datainsight\.cache\huggingface\token
Login successful


  0%|          | 10/1460238 [00:41<1688:18:16,  4.16s/it]


KeyboardInterrupt: 

## Final Pipeline

In [2]:
import os
import glob
import pandas as pd
import torch
from tqdm import tqdm

def run_full_pipeline_HOptimus_fivecrop(
    patch_data_dir: str,
    extractor,  # Instance of HOptimusExtractorFiveCrop
    non_white_csv: str,
    output_root: str
):
    # Load non-white metadata
    non_white_df = pd.read_csv(non_white_csv)
    valid_patch_names = set(non_white_df['patch_name'].astype(str))

    # Loop over slide directories
    slide_dirs = glob.glob(os.path.join(patch_data_dir, "*"))
    for slide_dir in tqdm(slide_dirs, desc="🧩 Processing slides", unit="slide", position=0):
        slide_name = os.path.basename(slide_dir)
        patch_feat_dir = os.path.join(output_root, slide_name)
        os.makedirs(patch_feat_dir, exist_ok=True)

        # Filter valid non-white patches
        all_patches = glob.glob(os.path.join(slide_dir, "*.png"))
        valid_patch_paths = [p for p in all_patches if os.path.basename(p) in valid_patch_names]

        if not valid_patch_paths:
            tqdm.write(f"  ❌ No valid non-white patches found for {slide_name}")
            continue

        # Skip already processed patches
        existing_feats = set(os.path.splitext(f.name)[0] for f in os.scandir(patch_feat_dir) if f.name.endswith(".pt"))
        to_process = [p for p in valid_patch_paths if os.path.splitext(os.path.basename(p))[0] not in existing_feats]

        if not to_process:
            tqdm.write(f"  ⏭️ Skipped: All {len(valid_patch_paths)} patches already processed for {slide_name}")
            continue

        # Patch-level tqdm bar
        for patch_path in tqdm(to_process, desc=f"🔄 {slide_name}", unit="patch", position=1, leave=False):
            patch_name = os.path.splitext(os.path.basename(patch_path))[0]
            save_path = os.path.join(patch_feat_dir, patch_name + ".pt")

            try:
                features = extractor.extract_fivecrop_features(patch_path)  # shape (5, D)
                torch.save(features, save_path)
            except Exception as e:
                tqdm.write(f"  ❌ Failed to process patch {patch_path}: {e}")
                continue

        tqdm.write(f"  ✅ Done: Processed {len(to_process)} new patches (total: {len(valid_patch_paths)}) for {slide_name}")

In [4]:
extractor = HOptimusExtractorFiveCrop()

run_full_pipeline_HOptimus_fivecrop(
    patch_data_dir=r"D:\Aamir Gulzar\KSA_project2\dataset\patch_data",
    extractor=extractor,
    non_white_csv=r"D:\Aamir Gulzar\KSA_project2\dataset\final_merged_without_white.csv",
    output_root=r"D:\Aamir Gulzar\KSA_project2\dataset\Features\H_Optimus_1_fivecrop_Updated"
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\datainsight\.cache\huggingface\token
Login successful


🧩 Processing slides:   0%|          | 1/417 [00:00<02:20,  2.95slide/s]

  ⏭️ Skipped: All 1878 patches already processed for TCGA-3L-AA1B_nonMSIH


🧩 Processing slides:   0%|          | 2/417 [00:02<08:53,  1.29s/slide]

  ⏭️ Skipped: All 4240 patches already processed for TCGA-4N-A93T_nonMSIH


...

  ✅ Done: Processed 5746 new patches (total: 5746) for TCGA-SS-A7HO_nonMSIH


🧩 Processing slides: 100%|█████████▉| 416/417 [11:05:19<11:46, 706.44s/slide]

  ✅ Done: Processed 5462 new patches (total: 5462) for TCGA-T9-A92H_nonMSIH


🧩 Processing slides: 100%|██████████| 417/417 [11:10:54<00:00, 96.53s/slide] 


  ✅ Done: Processed 4532 new patches (total: 4532) for TCGA-WS-AB45_MSIH


## Check .pt file

In [3]:
import torch
f = torch.load(r"D:\Aamir Gulzar\KSA_project2\dataset\Features\H_Optimus_1_fivecrop_Updated\TCGA-3L-AA1B_nonMSIH\TCGA-3L-AA1B_nonMSIH_x10288_y12512_patch00159.pt")
print(f.shape)       # Should be torch.Size([5, 1536])
print(f.element_size(), f.numel(), f.element_size() * f.numel() / 1024, "KB")

torch.Size([5, 1536])
4 7680 30.0 KB


In [4]:
import torch

def inspect_pt_file(pt_path):
    """Inspect the contents of a PyTorch .pt file"""
    print(f"Inspecting PT file: {pt_path}")
    print("=" * 50)

    data = torch.load(pt_path, map_location='cpu')

    if isinstance(data, dict):
        for key, value in data.items():
            print(f"Key: {key}")
            if isinstance(value, torch.Tensor):
                print(f"  Type: Tensor")
                print(f"  Shape: {tuple(value.shape)}")
                print(f"  Dtype: {value.dtype}")
                print("  First 10 values in each of the 5 rows:")
                for i in range(min(5, value.shape[0])):  # Only first 5 rows
                    row_values = value[i][:10].tolist()
                    print(f"    Row {i+1}: {row_values}")
            else:
                print(f"  Type: {type(value)}")
                print(f"  Value: {value}")
            print()
    elif isinstance(data, torch.Tensor):
        print(f"Single Tensor:")
        print(f"  Shape: {tuple(data.shape)}")
        print(f"  Dtype: {data.dtype}")
        print("  First 10 values in each of the 5 rows:")
        for i in range(min(5, data.shape[0])):
            row_values = data[i][:10].tolist()
            print(f"    Row {i+1}: {row_values}")
    else:
        print("The .pt file is not a dictionary or tensor. Type:", type(data))
        print("Value preview:", str(data)[:1500])  # Print partial value


pt_path = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\H_Optimus_1_fivecrop_Updated\TCGA-3L-AA1B_nonMSIH\TCGA-3L-AA1B_nonMSIH_x10288_y12512_patch00159.pt"
inspect_pt_file(pt_path)


Inspecting PT file: D:\Aamir Gulzar\KSA_project2\dataset\Features\H_Optimus_1_fivecrop_Updated\TCGA-3L-AA1B_nonMSIH\TCGA-3L-AA1B_nonMSIH_x10288_y12512_patch00159.pt
Single Tensor:
  Shape: (5, 1536)
  Dtype: torch.float32
  First 10 values in each of the 5 rows:
    Row 1: [-0.13186094164848328, 0.2290773093700409, -0.6069474220275879, -0.2261803150177002, -0.5075722932815552, 0.631453275680542, -0.595493733882904, 0.1406000703573227, -0.10133633762598038, 0.05652245134115219]
    Row 2: [-0.9243358373641968, 0.0937545970082283, -0.959202229976654, -0.7664600610733032, -0.04866240173578262, 0.4886153042316437, -1.2725995779037476, 0.1663188338279724, -0.7203095555305481, 0.5352378487586975]
    Row 3: [-0.47139784693717957, 0.1314208060503006, -0.6246311068534851, 0.09655484557151794, -0.1491871029138565, -0.6301625370979309, -0.5399565100669861, 0.13864348828792572, 0.0882948786020279, 0.48060446977615356]
    Row 4: [-0.43088388442993164, 0.19961830973625183, -1.2445062398910522, -1.

In [5]:
import torch
import os

def run_sanity_checks(tensor: torch.Tensor):
    print("Running sanity checks...\n")

    # 1. Shape
    if tensor.shape != (5, 1536):
        print(f"[❌] Shape mismatch: Expected (5, 1536), got {tuple(tensor.shape)}")
    else:
        print(f"[✅] Shape OK: {tuple(tensor.shape)}")

    # 2. Dtype
    if tensor.dtype not in [torch.float32, torch.float16]:
        print(f"[❌] Unexpected dtype: {tensor.dtype}")
    else:
        print(f"[✅] Dtype OK: {tensor.dtype}")

    # 3. NaN or Inf
    if not torch.isfinite(tensor).all():
        nan_count = torch.isnan(tensor).sum().item()
        inf_count = torch.isinf(tensor).sum().item()
        print(f"[❌] Invalid values found — NaNs: {nan_count}, Infs: {inf_count}")
    else:
        print(f"[✅] No NaN or Inf values")

    # 4. Value range
    min_val = tensor.min().item()
    max_val = tensor.max().item()
    print(f"[ℹ️] Value range: min={min_val:.4f}, max={max_val:.4f}")
    if abs(min_val) > 1e6 or abs(max_val) > 1e6:
        print("[⚠️] Warning: Value range is unusually large.")

    # 5. L2 norm (per row)
    norms = tensor.norm(p=2, dim=1)
    print(f"[ℹ️] L2 Norms (per row): {norms.tolist()}")
    if (norms == 0).any():
        print("[❌] Zero vector detected (row with all zeros)")
    else:
        print("[✅] All rows have non-zero magnitude")

    # 6. Row consistency (std across rows)
    row_std = torch.std(tensor, dim=1)
    print(f"[ℹ️] Row-wise std dev: {row_std.tolist()}")
    if (row_std == 0).any():
        print("[⚠️] Warning: At least one row is constant (no variance)")

    print("\nSanity check completed.")

# Example usage
pt_path = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\H_Optimus_1_fivecrop_Updated\TCGA-3L-AA1B_nonMSIH\TCGA-3L-AA1B_nonMSIH_x10288_y12512_patch00159.pt"
data = torch.load(pt_path, map_location='cpu')

# If it's a dict with a tensor inside
if isinstance(data, dict):
    for key in data:
        if isinstance(data[key], torch.Tensor) and data[key].shape == (5, 1536):
            run_sanity_checks(data[key])
            break
else:
    run_sanity_checks(data)


Running sanity checks...

[✅] Shape OK: (5, 1536)
[✅] Dtype OK: torch.float32
[✅] No NaN or Inf values
[ℹ️] Value range: min=-3.2731, max=3.1491
[ℹ️] L2 Norms (per row): [31.616098403930664, 32.858863830566406, 32.53364562988281, 33.56842041015625, 32.72332763671875]
[✅] All rows have non-zero magnitude
[ℹ️] Row-wise std dev: [0.8069582581520081, 0.8386797308921814, 0.8303708434104919, 0.8567925095558167, 0.8351950645446777]

Sanity check completed.
